### <mark>**✅Assign Stock**

#### **Library**

In [46]:
import pandas as pd
from datetime import datetime

#### **Define**

In [47]:
# 1. 데이터 불러오기
stock = pd.read_csv('./data/stock.csv', encoding='utf-8-sig')
sales = pd.read_csv('./data/sales.csv', encoding='cp949')
sku = pd.read_csv('./data/sku.csv', encoding='cp949')

# 2. 자재 재고 딕셔너리화
stock = dict(zip(stock['sku'], stock['stock']))

# 3. 판매량 정보 merge
sales_sku = sku.merge(sales, on='sku', how='left')

# 4. TODAY
today = datetime.today().strftime('%Y%m%d')

#### **Assign**

In [48]:
# 4. 자재 사용량 계산
sales_sku['material_usage'] = sales_sku['quantity'] * sales_sku['sales']
total_usage = sales_sku.groupby('material')['material_usage'].sum().to_dict()

# 5. 비중 기반 자재 할당량 계산
sales_sku['material_ratio'] = sales_sku.apply(
    lambda row: row['material_usage'] / total_usage[row['material']] if total_usage[row['material']] > 0 else 0,
    axis=1
)
sales_sku['allocated_material'] = sales_sku.apply(
    lambda row: row['material_ratio'] * stock[row['material']],
    axis=1
)

# 6. SKU별 할당 가능한 주문 수량 계산
sku_group = sales_sku.groupby(['code', 'sku', 'material'])[['quantity', 'allocated_material']].sum().reset_index()
sku_group['max_units'] = sku_group['allocated_material'] // sku_group['quantity']
final_units = sku_group.groupby(['code', 'sku'])['max_units'].min().astype(int)

# 7. 자재별 실제 사용량 계산
sku_group = sku_group.merge(final_units.rename('allocated_units'), on=['code', 'sku'])
sku_group['used_stock'] = sku_group['allocated_units'] * sku_group['quantity']

# 8. 자재별 할당량 계산
material_usage = sku_group.pivot(index=['code', 'sku'], columns='material', values='used_stock').fillna(0).astype(int)
material_usage['assign'] = final_units.values

# 9. 결과 정리
material_usage = material_usage.reset_index().set_index(['code', 'sku'])
material_usage.index.names = ['code', 'sku']
assign_stock = material_usage[[col for col in material_usage.columns if col != 'assign'] + ['assign']]

In [49]:
# 10. TO CSV
assign_stock.to_csv(f'./result/{today}_stock.csv', encoding='utf-8-sig')

#### **T1**

In [50]:
# 1. 데이터 불러오기
today = datetime.today().strftime('%Y%m%d')
assign_stock = pd.read_csv(f'./result/{today}_stock.csv', encoding='utf-8-sig')
t1 = pd.read_csv('./result/t1.csv', encoding='cp949')

In [51]:
# 2. SKU를 기준으로 병합
df = assign_stock.merge(t1, on='sku', how='left')

# 3. NaN 판매수량은 0으로 처리
df['t1'] = df['t1'].fillna(0)

# 4. 그린/퍼플 자재 사용량 계산
#    (SKU 한 개당 자재 수량 비율 = green/assign, purple/assign)
df['green_used'] = df['t1'] * (df['green'] / df['assign'])
df['purple_used'] = df['t1'] * (df['purple'] / df['assign'])

# 5. 총 사용량 계산
green_total_used = df['green_used'].sum()
purple_total_used = df['purple_used'].sum()

# 6. 기존 총 재고량 계산 (assign_stock 기준 전체 green, purple 합계)
green_total_stock = assign_stock['green'].sum()
purple_total_stock = assign_stock['purple'].sum()

# 7. 잔여 자재량 계산
green_remaining = green_total_stock - green_total_used
purple_remaining = purple_total_stock - purple_total_used

In [52]:
stock_remaining_df = pd.DataFrame({
    'sku': ['green', 'purple'],
    'stock': [int(green_remaining), int(purple_remaining)]
})

# 10. CSV로 저장
stock_remaining_df.to_csv('./data/stock.csv', index=False, encoding='utf-8-sig')


#### **2page**

In [53]:
# 1. 데이터 불러오기
stock = pd.read_csv('./data/stock.csv', encoding='utf-8-sig')
sales = pd.read_csv('./data/sales.csv', encoding='cp949')
sku = pd.read_csv('./data/sku.csv', encoding='cp949')

# 2. 자재 재고 딕셔너리화
stock = dict(zip(stock['sku'], stock['stock']))

# 3. 판매량 정보 merge
sales_sku = sku.merge(sales, on='sku', how='left')

# 4. TODAY
today = datetime.today().strftime('%Y%m%d')

# 4. 자재 사용량 계산
sales_sku['material_usage'] = sales_sku['quantity'] * sales_sku['sales']
total_usage = sales_sku.groupby('material')['material_usage'].sum().to_dict()

# 5. 비중 기반 자재 할당량 계산
sales_sku['material_ratio'] = sales_sku.apply(
    lambda row: row['material_usage'] / total_usage[row['material']] if total_usage[row['material']] > 0 else 0,
    axis=1
)
sales_sku['allocated_material'] = sales_sku.apply(
    lambda row: row['material_ratio'] * stock[row['material']],
    axis=1
)

# 6. SKU별 할당 가능한 주문 수량 계산
sku_group = sales_sku.groupby(['code', 'sku', 'material'])[['quantity', 'allocated_material']].sum().reset_index()
sku_group['max_units'] = sku_group['allocated_material'] // sku_group['quantity']
final_units = sku_group.groupby(['code', 'sku'])['max_units'].min().astype(int)

# 7. 자재별 실제 사용량 계산
sku_group = sku_group.merge(final_units.rename('allocated_units'), on=['code', 'sku'])
sku_group['used_stock'] = sku_group['allocated_units'] * sku_group['quantity']

# 8. 자재별 할당량 계산
material_usage = sku_group.pivot(index=['code', 'sku'], columns='material', values='used_stock').fillna(0).astype(int)
material_usage['assign'] = final_units.values

# 9. 결과 정리
material_usage = material_usage.reset_index().set_index(['code', 'sku'])
material_usage.index.names = ['code', 'sku']
assign_stock = material_usage[[col for col in material_usage.columns if col != 'assign'] + ['assign']]

# 10. TO CSV
assign_stock.to_csv(f'./result/{today}_stock.csv', encoding='utf-8-sig')